In [1]:
#import packages
import datetime as dt 
import lxml
from lxml import html, etree
import requests
import numpy as np
import pandas as pd
import pandas_datareader
from pandas_datareader import data as pdr
import time
from time import sleep
import string
import yfinance as yf
from bs4 import BeautifulSoup
import urllib
import io
import os
import pandasql
from pandasql import sqldf
import re
import gc
import zipfile
from zipfile import ZipFile
from io import StringIO
import sys

from os import listdir
from os.path import isfile, join
import psutil

from collections import Counter
import linecache
import tracemalloc

import reshape

import import_ipynb
import current_scraper
from current_scraper import *


c:\users\daniel\appdata\local\programs\python\python38-32\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


importing Jupyter notebook from current_scraper.ipynb


In [2]:
os.getcwd()

'D:\\GitHub\\finance'

In [3]:
def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [4]:
#get ticker, company name and CIK numbers 
ticker_url = "https://www.sec.gov/include/ticker.txt"
ticker_request = requests.get(ticker_url).content
ticker_df = pd.read_csv(io.StringIO(ticker_request.decode('utf-8')),sep="\t")
ticker_df.columns = ['ticker', 'cik']
ticker_df["ticker"] = ticker_df["ticker"].str.lower()


#get sp500 tickers 
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500 = pd.DataFrame(sp500[0]['Symbol'])
sp500.columns = ['ticker']
sp500["ticker"] = sp500["ticker"].str.lower()
#print(sp500)

#get cik of sp 500
sample = pd.merge(ticker_df,sp500,on='ticker')
print(sample )

    ticker      cik
0      aal     6201
1      aap  1158449
2     aapl   320193
3     abbv  1551152
4      abc  1140859
..     ...      ...
495    yum  1041061
496    zbh  1136869
497   zbra   877212
498   zion   109380
499    zts  1555280

[500 rows x 2 columns]


In [ ]:
#get tickers in the sp 500 and get easy stats on them
#get avg price by year, and current price of stock 
def get_easy_stats(df):
    
    easy_df = []
    
    start_dt = dt.datetime(2013,1,1)
    end_dt = dt.datetime(2019,12,31)
    
    for index, row in df.iterrows():
        
        try: 
        
            sleep(0.2)
            #data_ts = pdr.get_data_yahoo(row['ticker'], start_dt, end_dt)
            data_ts = yf.download(row['ticker'], start= start_dt, end= end_dt, progress=False)
            #print(data_ts)
                                      
            data_ts['year'] = data_ts.index.year
            #get average price per share 
            agg_df = data_ts.groupby('year', as_index=False)['Adj Close'].mean()
            agg_df['ticker'] = row['ticker']
            agg_df.columns = ['year','avg_price', 'ticker']
            agg_df['year']=  agg_df['year'].astype(str)
    #       print(agg_df)
            current_price = pdr.get_data_yahoo(row['ticker']).last('1D')['Adj Close'][0].astype(int)

            #turn agg_df wide  for price rows
            wide_df = agg_df.pivot(index='ticker', columns='year', values=['avg_price'])

            wide_df.columns = list(map("".join, wide_df.columns))
            wide_df['current_price'] = current_price
            print(wide_df)

            easy_df.append(wide_df)
            del agg_df
            del wide_df
        
        except Exception as ex:
            print(row['ticker'] + " failed")
            print(ex)
            
    
    easy_df = pd.concat(easy_df)
    #easy_df.to_csv("easy_stats_sp500.csv")
    return(easy_df)
#function end

start = time.time()
#run to generate easy stat csv
easy_stat_df = get_easy_stats(sample)
easy_stat_df.to_csv("easy_stats_sp500.csv")
print(easy_stat_df)

end = time.time()
print("finished getting easy stats for sp500")
print(end - start)



In [ ]:
#read in the easy stats of sp500
#easy_stat_df.to_csv("easy_stats_sp500.csv")
easy_stat_df = pd.read_csv("easy_stats_sp500.csv")
print(easy_stat_df)

In [ ]:
#set up script to traverse directory of quarterly submissions
def generate_finance_table_csv():

    #store all finances in a table
    finance_list = []

    #go into directory with data
    file_os = 'D:\\Finance Data'
    
    
    code_os = 'D:\\GitHub\\finance'
    start = time.time()
    
    #go into file directory
    os.chdir(file_os)
    
    for i in os.listdir():
        file_path = i
        #if its a quarterly filing data, parse it
        if 'q1' in file_path or 'q2' in file_path or 'q3' in file_path or 'q4' in file_path:
            try:
                tracemalloc.start()
                print("starting: " + file_path)

                sub = pd.read_csv(file_path + '/' + 'sub.txt', sep = '\t', encoding = "ISO-8859-1", iterator = True, chunksize =100000)
                sub = pd.concat(sub)
                #clean up form column and filter to just 10-K submissions
                sub['form'] = sub['form'].replace(np.nan, '', regex=True)
                sub = sub[sub['form'] == '10-K']
                #filter to the s&p500
                sub = pd.merge(sub,sample, on = ['cik'])

                pre = pd.read_csv(file_path + '/' + 'pre.txt', sep = '\t', encoding = "ISO-8859-1", chunksize =1000000) 
                num = pd.read_csv(file_path + '/' + 'num.txt', sep = '\t', encoding = "ISO-8859-1",  chunksize =100000)

                print("joining num to sub")
                num_sub = pd.DataFrame()              
                #join in numbers to submissions in chunks
                for chunks in num: 
                    #print(chunks)
                    num_sub = pd.concat([num_sub, sub.merge(chunks, on=['adsh'])])

                #no longer need sub
                del sub

                print("joining pre to numsub")
                nsp = pd.DataFrame()
                #add in plabel and stmt info
                for chunks in pre:
                    #print(chunks)
                    nsp = pd.concat([nsp, num_sub.merge(chunks, on=['adsh', 'tag', 'version'])])
                    nsp = nsp[nsp['stmt'].isin(['BS', 'IS', 'CF', 'CI', 'EQ'])]
                # delete files no longer needed
                del num_sub
                del pre
                del num


                #print(nsp.columns)

                nsp = nsp[['name', 'sic', 'fye', 'form', 'period', 
                           'fy', 'fp', 'filed','ticker', 'cik','ddate', 
                           'qtrs', 'uom', 'value', 'adsh','stmt',  'tag', 
                           'version', 'plabel']]

                #add finances to master table
                finance_list.append(nsp)
                del nsp
                print("processed_data: " + file_path)
                gc.collect()
                snapshot = tracemalloc.take_snapshot()
                display_top(snapshot)
            except Exception as ex:
                print("failed: " + file_path)
                print(str(ex))
    
    #switch back to github directory 
    os.chdir(code_os)
    
    
    finance_table = pd.concat(finance_list)
    del finance_list 
    finance_table.to_csv("total_finance_table.csv")
    end = time.time()
    print("finished traversing files in X secs")
    print(end - start)
    #return(finance_table)

In [ ]:
# #run function to generate finance table csv
generate_finance_table_csv()

In [ ]:
#main function goes through and tries to parse out the historical data we need: VERY LONG 
def parse_data(df):
    gc.collect()
    ############################################################
    #process data a bit 
    
    #turn things into an integer
    df['qtrs'] = df.qtrs.astype(int)
    df['period'] = df.period.astype(int)
    df['fy'] = df.fy.astype(int)
    df['fye'] = df.fye.astype(int)
    df['ddate'] = df.ddate.astype(int)
    df['ddate_prev'] = df.ddate.astype(int) + 10000
    
    #turn things to strings
    df['period'] = df['period'].astype(str)
    df['fye'] = df['fye'].astype(str)
    df['fy'] = df['fy'].astype(str)
    df['ddate'] = df['ddate'].astype(str)
    df['ddate_prev'] = df['ddate_prev'].astype(str)
    #pad fye with a leading 0 
    df['fye'] = df['fye'].apply(lambda x: x.zfill(4))
    
    #fill nas inplace = true changes the df directly
    df['period'].fillna('', inplace=True)
    df['fy'].fillna('', inplace=True)
    df['fye'].fillna('', inplace=True)
    df['plabel'].fillna('', inplace=True)
    df['ddate'].fillna('', inplace=True)
    df['ddate_prev'].fillna('', inplace=True)
    
    #create a joinable key
    df['id'] = df['fy'].astype(str) + '_' + df['cik'].astype(str)

    #get distinct company and years
    key_df = pd.DataFrame(df.id.unique(), columns = ['id'])
    gc.collect()
    
    #######################################################################################################
    #define tags
    
    #for shares outstanding calc (all qtrs = 4, income stmt)
    shares_outstanding_tags = ['WeightedAverageNumberOfDilutedSharesOutstanding']
    net_income_tags = ['NetIncomeLoss']
    eps_tags = ['EarningsPerShareDiluted']
    
    # for fcfe calculation  (cash flow stmt)
    cfo_tags = ['NetCashProvidedByUsedInOperatingActivities','NetCashProvidedByUsedInOperatingActivitiesContinuingOperations']
    cfo_plabels = ['Cash generated by operating activities']
    debt_repayments_tags= ['RepaymentsOfDebt', 'RepaymentsOfLongTermDebt', 'RepaymentsOfOtherLongTermDebt',
                           'RepaymentsOfDebtAndCapitalLeaseObligations','RepaymentsOfLongTermDebtAndCapitalLeaseObligations',
                          'RepaymentsOfLongTermDebtAndCapitalSecurities', 'RepaymentsOfLongTermCapitalLeaseObligations']
    debt_repayments_plabels = ['Repayments of long-term debt and finance lease obligations', 
                               'Maturities of long-term debt', 'Repayments of recourse debt']
    debt_proceeds_tags = ['ProceedsFromIssuanceOfDebt', 'ProceedsFromIssuanceOfLongTermDebt']
    
    #for capital expenditure calculations (cash flow statement)
    capex_tags = ['PaymentsToAcquireProductiveAssets']
    capex_plabels = ['Capital expenditures']
    ppe_purchase_tags = ['PaymentsToAcquirePropertyPlantAndEquipment', 'PaymentsToAcquireProductiveAssets']
    ppe_sale_tags = ['ProceedsFromSaleOfPropertyPlantAndEquipment', 
                     'ProceedsFromSaleOfPropertyPlantAndEquipmentAndSaleLeaseBackTransactions']
    #qtrs = 0; for ppe on balance sheet
    ppe_tags = ['PropertyPlantAndEquipmentNet']

    #for ebitda calculation
    revenue_tags = ['Revenues', 'RevenueFromContractWithCustomerExcludingAssessedTax', 'SalesRevenueNet','SalesRevenueServicesNet']
    revenue_plabels = ['Net Sales']
    
    operating_income_tags = ['OperatingIncomeLoss']
    dep_amort_tags = ['DepreciationDepletionAndAmortization', 'Depreciation']
    dep_amort_plabels = ['Depreciation and amortization']
    
    #for enterprise value calculation
    ltd_tags = ['LongTermDebtNoncurrent', 'LongTermDebtAndCapitalLeaseObligations']
    std_tags = ['LongTermDebtCurrent', 'LongTermDebtAndCapitalLeaseObligationsCurrent']
    cash_tags = ['CashAndCashEquivalentsAtCarryingValue', 'Cash']
    


    
    #######################################################################################################
    #pull rows
    
#shares outstanding
    shares_outstanding_rows = df[
        (df['tag'].isin(shares_outstanding_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.uom.str.contains('shares',na=False))
    ]

    shares_outstanding_agg = shares_outstanding_rows.groupby(['id'], as_index=False)['value'].max()  
    shares_outstanding_agg.columns = ['id', 'shares_outstanding']
    del shares_outstanding_rows
#net income
    net_income_rows = df[
        (df['tag'].isin(net_income_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    net_income_agg = net_income_rows.groupby(['id'], as_index=False)['value'].max()  
    net_income_agg.columns = ['id', 'net_income']
    del net_income_rows
#eps
    eps_rows = df[
        (df['tag'].isin(eps_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    eps_agg = eps_rows.groupby(['id'], as_index=False)['value'].max()  
    eps_agg.columns = ['id', 'eps']
    del eps_rows
#cash flow from operations    
    cfo_rows = df[
        (df['tag'].isin(cfo_tags) | df['plabel'].isin(cfo_plabels))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    cfo_agg = cfo_rows.groupby(['id'], as_index=False)['value'].max()  
    cfo_agg.columns = ['id', 'cfo']
    del cfo_rows
#debt repayment 
    debt_repayments_rows = df[
        (df['tag'].isin(debt_repayments_tags)| df['plabel'].isin(debt_repayments_plabels))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    debt_repayments_agg = debt_repayments_rows.groupby(['id'], as_index=False)['value'].max()  
    debt_repayments_agg.columns = ['id', 'debt_repayments']
    del debt_repayments_rows
#debt proceeds 
    debt_proceeds_rows = df[
        (df['tag'].isin(debt_proceeds_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    debt_proceeds_agg = debt_proceeds_rows.groupby(['id'], as_index=False)['value'].max()  
    debt_proceeds_agg.columns = ['id', 'debt_proceeds']
    del debt_proceeds_rows
#cap ex
    capex_rows = df[
        (df['tag'].isin(capex_tags)| df['plabel'].isin(capex_plabels))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    capex_agg = capex_rows.groupby(['id'], as_index=False)['value'].max()  
    capex_agg.columns = ['id', 'capex']
    del capex_rows
#ppe purchase
    ppe_purchase_rows = df[
        (df['tag'].isin(ppe_purchase_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    ppe_purchase_agg = ppe_purchase_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_purchase_agg.columns = ['id', 'ppe_purchase']
    del ppe_purchase_rows
#ppe sale 
    ppe_sale_rows = df[
        (df['tag'].isin(ppe_sale_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    ppe_sale_agg = ppe_sale_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_sale_agg.columns = ['id', 'ppe_sale']
    del ppe_sale_rows
#ppe , qtrs = 0; for ppe on balance sheet
    ppe_rows = df[
        (df['tag'].isin(ppe_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 0)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    ppe_agg = ppe_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_agg.columns = ['id', 'ppe']
    del ppe_rows
#get PPE of the year before
    ppe_prev_rows = df[
        (df['tag'].isin(ppe_tags))
        & (df['ddate_prev'] == (df['fy'] + df['fye']))
        & (df.qtrs == 0)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    ppe_prev_agg = ppe_prev_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_prev_agg.columns = ['id', 'ppe_prev']
    del ppe_prev_rows
#revenue 
    revenue_rows = df[
        (df['tag'].isin(revenue_tags) | df['plabel'].isin(revenue_plabels))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    revenue_agg = revenue_rows.groupby(['id'], as_index=False)['value'].max()  
    revenue_agg.columns = ['id', 'revenue']
    del revenue_rows
#operating income
    operating_income_rows = df[
        (df['tag'].isin(operating_income_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    operating_income_agg = operating_income_rows.groupby(['id'], as_index=False)['value'].max()  
    operating_income_agg.columns = ['id', 'operating_income']
    del operating_income_rows
#depreciation and amortization
    dep_amort_rows = df[
        (df['tag'].isin(dep_amort_tags) | df['plabel'].isin(dep_amort_plabels))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    dep_amort_agg = dep_amort_rows.groupby(['id'], as_index=False)['value'].max()  
    dep_amort_agg.columns = ['id', 'dep_amort']
    del dep_amort_rows

#     #get long term debt on balance sheet
    ltd_rows = df[
        (df['tag'].isin(ltd_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    ltd_agg = ltd_rows.groupby(['id'], as_index=False)['value'].max()  
    ltd_agg.columns = ['id', 'ltd']
    del ltd_rows
#     #get short term debt on balance sheet 
    std_rows = df[
        (df['tag'].isin(std_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    std_agg = std_rows.groupby(['id'], as_index=False)['value'].max()  
    std_agg.columns = ['id', 'std']
    del std_rows
#     #get cash and cash equivalents
    cash_rows = df[
        (df['tag'].isin(cash_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    cash_agg = cash_rows.groupby(['id'], as_index=False)['value'].max()  
    cash_agg.columns = ['id', 'cash']
    del cash_rows
    
#     #######################################################################################################
    #add financials to key_df
    merged_df = key_df.merge(shares_outstanding_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(net_income_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(eps_agg, on = ['id'], how = "left")
    
    merged_df = merged_df.merge(cfo_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(debt_proceeds_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(debt_repayments_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(capex_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_purchase_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_sale_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_prev_agg, on = ['id'], how = "left")
    
    merged_df = merged_df.merge(revenue_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(operating_income_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(dep_amort_agg, on = ['id'], how = "left")
    
    merged_df = merged_df.merge(ltd_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(std_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(cash_agg, on = ['id'], how = "left")
    
    
#     #######################################################################################################
#     #delete unneeded things 
    del shares_outstanding_agg
    del net_income_agg
    del eps_agg
    del cfo_agg
    del debt_proceeds_agg
    del debt_repayments_agg
    del capex_agg
    del ppe_purchase_agg
    del ppe_sale_agg
    del ppe_agg
    del ppe_prev_agg
    del revenue_agg
    del operating_income_agg
    del dep_amort_agg
    return(merged_df)



In [ ]:
def first_value(x):
    if x.first_valid_index() is None:
        return None
    else:
        return x[x.first_valid_index()]
    
def last_value(x):
    if x.last_valid_index() is None:
        return None
    else:
        return x[x.last_valid_index()]
    
#function that runs yahoo querying on sample tickers
def get_yahoo_info():
    yahoo_info = get_info(sample['ticker'])
    yahoo_info.to_csv("yahoo_info.csv")
    return(yahoo_info)

In [ ]:
# process the data in chunks first, return ticker level dataset 
def init_process_data():
    
    processed_list = []
    
    chunksize = 10 ** 5
    for chunk in pd.read_csv("total_finance_table.csv", chunksize=chunksize):  
        #for each chunk, get finance info on a company, yearly level
        processed_list.append(parse_data(chunk))
#         break
    #turn list of dataframes into one dataframe
    processed_table = pd.concat(processed_list)
    

    #split id back into year and cik
    processed_table[['year','cik']] = processed_table.id.str.split(pat = "_", expand=True) 
    #print("printing pre-aggregated data")
    #print(processed_table)
    del processed_list
    #######################################################################################################
    #aggregate up across chunks 
    agg_df = sqldf(
    """
    SELECT 
    *, 
    fcfe/NULLIF(shares_outstanding_coalesced,0) AS fcfe_per_share
    FROM 
    (
        SELECT
        *, 
        cfo - capex_coalesced +  IFNULL(debt_repayments,0) AS fcfe
        FROM 
        (
             SELECT 
            *,
            IFNULL(operating_income,0) + IFNULL(dep_amort,0) AS ebitda, 

            CASE WHEN shares_outstanding IS NULL THEN net_income/NULLIF(eps,0) 
                 ELSE shares_outstanding 
                 END AS shares_outstanding_coalesced, 

            CASE WHEN capex IS NOT NULL THEN capex 
                 WHEN ppe_purchase IS NOT NULL THEN ppe_purchase - IFNULL(ppe_sale,0)
                 WHEN ppe IS NOT NULL THEN ppe - IFNULL(ppe_prev,0) 
                 END AS capex_coalesced, 

            IFNULL(debt_proceeds,0) - IFNULL(debt_repayments,0) AS net_debt_proceeds, 
            
            IFNULL(ltd,0) + IFNULL(std,0) AS total_debt 
            
            FROM 
            (
                SELECT  
                year,
                CAST(cik AS INT64) as cik,
                MAX(shares_outstanding) AS shares_outstanding,
                MAX(net_income) AS net_income,
                MAX(eps) AS eps,
                MAX(cfo) AS cfo,
                MAX(debt_proceeds) AS debt_proceeds,
                MAX(debt_repayments) AS debt_repayments,
                MAX(capex) AS capex,
                MAX(ppe_purchase) AS ppe_purchase,
                MAX(ppe_sale) AS ppe_sale,
                MAX(ppe) AS ppe,
                MAX(ppe_prev) AS ppe_prev, 
                MAX(revenue) AS revenue,
                MAX(operating_income) AS operating_income,
                MAX(dep_amort) AS dep_amort, 
                MAX(ltd) AS ltd,
                MAX(std) AS std, 
                MAX(cash) AS cash 
                FROM 
                processed_table 
                GROUP BY     
                year,
                cik
                ORDER BY 2 DESC
            )
            )
            
    )
    
    """)
    
    del processed_table 
    #print(agg_df.columns)
    
#     #######################################################################################################
    # turn long data wide 
    wide_df = agg_df.pivot(index='cik', columns='year', values=['shares_outstanding', 'net_income', 'eps', 'cfo',
       'debt_proceeds', 'debt_repayments', 'capex', 'ppe_purchase', 'ppe_sale',
       'ppe', 'ppe_prev', 'revenue', 'operating_income', 'dep_amort', 'ebitda',
       'shares_outstanding_coalesced', 'capex_coalesced', 'net_debt_proceeds',
       'fcfe', 'fcfe_per_share', 'ltd', 'std', 'cash'])
    
    del agg_df
    
#     ####################################################################################################### 
    #fix columns
    wide_df.columns = list(map("".join, wide_df.columns))
    
    #add in tickers
    wide_df = wide_df.merge(sample, on = ['cik'])
    
    #drop weird columns 
    wide_df = wide_df[wide_df.columns.drop(list(wide_df.filter(regex='1218')))]
    wide_df = wide_df[wide_df.columns.drop(list(wide_df.filter(regex='2020')))]
    
    #read in easy stats csv
    easy_stat_df = pd.read_csv("easy_stats_sp500.csv")
    #read in yahoo info csv
    yahoo_info = pd.read_csv("yahoo_info.csv")
    
    
    #join in easy stats
    wide_df = wide_df.merge(easy_stat_df, on = 'ticker', how = 'left')
    wide_df = wide_df.merge(yahoo_info, on = 'ticker', how = 'left')
    
    
    ####################################################################################################### 
    # add dataquality columns - 
    #WTK: how many columns are available? first year, last year of data, years of data

    revenue_cols =  [col for col in wide_df.columns if 'revenue' in col]
    revenue_df = wide_df.loc[:,revenue_cols]
    #make a copy to avoid setting warnings
    copy_df = revenue_df.copy(deep=True) 
    revenue_df.loc[:,'revenue_last_value'] = copy_df.apply(last_value, axis=1)
    revenue_df.loc[:,'revenue_last_year'] = copy_df.apply(lambda x: x.last_valid_index(), axis=1)
    revenue_df.loc[:,'revenue_first_value'] = copy_df.apply(first_value, axis=1)
    revenue_df.loc[:,'revenue_first_year'] = copy_df.apply(lambda x: x.first_valid_index(), axis=1)
    revenue_df.loc[:,'revenue_years_avail'] = copy_df.apply(lambda x: x.count(), axis =1)
    #massage year fields  
    revenue_df[['revenue_first_year', 'revenue_last_year']] = revenue_df[['revenue_first_year', 'revenue_last_year']].fillna(value=0)
    revenue_df["revenue_first_year"] = revenue_df["revenue_first_year"].str.replace("revenue", "")
    revenue_df["revenue_first_year"].fillna(0, inplace = True)
    revenue_df["revenue_first_year"] = revenue_df["revenue_first_year"].astype(int, errors = "ignore")
    revenue_df["revenue_last_year"] = revenue_df["revenue_last_year"].str.replace("revenue", "")
    revenue_df["revenue_last_year"].fillna(0, inplace = True)
    revenue_df["revenue_last_year"] = revenue_df["revenue_last_year"].astype(int, errors = "ignore")
    #get years between first and lasts date 
    #print(revenue_df.dtypes)
    revenue_df['revenue_years_between'] = revenue_df['revenue_last_year'] - revenue_df['revenue_first_year']    
    #add ticker back
    revenue_df.loc[:,'ticker'] = wide_df['ticker']
    revenue_df = revenue_df[[
        'ticker',
        'revenue_last_value',
        'revenue_last_year',
        'revenue_first_value',
        'revenue_first_year',
        'revenue_years_avail',
        'revenue_years_between'
    ]]
    #print(revenue_df)
    
    #repeat above structure for shares and fcfe_modded
    ####################################################################################################### 
    shares_outstanding_cols =  [col for col in wide_df.columns if 'shares_outstanding_coalesced' in col]
    shares_outstanding_df = wide_df.loc[:,shares_outstanding_cols]
    #make a copy to avoid setting warnings
    copy_df = shares_outstanding_df.copy(deep=True) 
    shares_outstanding_df.loc[:,'shares_outstanding_last_value'] = copy_df.apply(last_value, axis=1)
    shares_outstanding_df.loc[:,'shares_outstanding_last_year'] = copy_df.apply(lambda x: x.last_valid_index(), axis=1)
    shares_outstanding_df.loc[:,'shares_outstanding_first_value'] = copy_df.apply(first_value, axis=1)
    shares_outstanding_df.loc[:,'shares_outstanding_first_year'] = copy_df.apply(lambda x: x.first_valid_index(), axis=1)
    shares_outstanding_df.loc[:,'shares_outstanding_years_avail'] = copy_df.apply(lambda x: x.count(), axis =1)
    #massage year fields  
    shares_outstanding_df[['shares_outstanding_first_year', 'shares_outstanding_last_year']] = shares_outstanding_df[['shares_outstanding_first_year', 'shares_outstanding_last_year']].fillna(value=0)
    shares_outstanding_df["shares_outstanding_first_year"] = shares_outstanding_df["shares_outstanding_first_year"].str.replace("shares_outstanding_coalesced", "")
    shares_outstanding_df["shares_outstanding_first_year"].fillna(0, inplace = True)
    shares_outstanding_df["shares_outstanding_first_year"] = shares_outstanding_df["shares_outstanding_first_year"].astype(int, errors = "ignore")
    shares_outstanding_df["shares_outstanding_last_year"] = shares_outstanding_df["shares_outstanding_last_year"].str.replace("shares_outstanding_coalesced", "")
    shares_outstanding_df["shares_outstanding_last_year"].fillna(0, inplace = True)
    shares_outstanding_df["shares_outstanding_last_year"] = shares_outstanding_df["shares_outstanding_last_year"].astype(int, errors = "ignore")
    #print(shares_outstanding_df.dtypes)
    #get years between first and lasts date 
    shares_outstanding_df['shares_outstanding_years_between'] = shares_outstanding_df['shares_outstanding_last_year'] - shares_outstanding_df['shares_outstanding_first_year']    
    #add ticker back
    shares_outstanding_df.loc[:,'ticker'] = wide_df['ticker'] 
    shares_outstanding_df = shares_outstanding_df[[
        'ticker',
        'shares_outstanding_last_value',
        'shares_outstanding_last_year',
        'shares_outstanding_first_value',
        'shares_outstanding_first_year',
        'shares_outstanding_years_avail',
        'shares_outstanding_years_between'
    ]]
    #print(shares_outstanding_df)
    
    ####################################################################################################### 
    fcfe_cols =  [col for col in wide_df.columns if 'fcfe2' in col]
    fcfe_df = wide_df.loc[:,fcfe_cols]
    #make a copy to avoid setting warnings
    copy_df = fcfe_df.copy(deep=True) 
    fcfe_df.loc[:,'fcfe_last_value'] = copy_df.apply(last_value, axis=1)
    fcfe_df.loc[:,'fcfe_last_year'] = copy_df.apply(lambda x: x.last_valid_index(), axis=1)
    fcfe_df.loc[:,'fcfe_first_value'] = copy_df.apply(first_value, axis=1)
    fcfe_df.loc[:,'fcfe_first_year'] = copy_df.apply(lambda x: x.first_valid_index(), axis=1)
    fcfe_df.loc[:,'fcfe_years_avail'] = copy_df.apply(lambda x: x.count(), axis =1)
    #massage year fields  
    fcfe_df[['fcfe_first_year', 'fcfe_last_year']] = fcfe_df[['fcfe_first_year', 'fcfe_last_year']].fillna(value=0)
    fcfe_df["fcfe_first_year"] = fcfe_df["fcfe_first_year"].str.replace("fcfe", "")
    fcfe_df["fcfe_first_year"].fillna(0, inplace = True)
    fcfe_df["fcfe_first_year"] = fcfe_df["fcfe_first_year"].astype(int, errors = "ignore")
    fcfe_df["fcfe_last_year"] = fcfe_df["fcfe_last_year"].str.replace("fcfe", "")
    fcfe_df["fcfe_last_year"].fillna(0, inplace = True)
    fcfe_df["fcfe_last_year"] = fcfe_df["fcfe_last_year"].astype(int, errors = "ignore")
    #print(fcfe_df.dtypes)
    #get years between first and lasts date 
    fcfe_df['fcfe_years_between'] = fcfe_df['fcfe_last_year'] - fcfe_df['fcfe_first_year']    
    #add ticker back
    fcfe_df.loc[:,'ticker'] = wide_df['ticker']   
    fcfe_df = fcfe_df[[
        'ticker',
        'fcfe_last_value',
        'fcfe_last_year',
        'fcfe_first_value',
        'fcfe_first_year',
        'fcfe_years_avail',
        'fcfe_years_between'
    ]]
    #print(fcfe_df)
    
    ####################################################################################################### 
    fcfe_per_share_cols =  [col for col in wide_df.columns if 'fcfe_per_share' in col]
    fcfe_per_share_df = wide_df.loc[:,fcfe_per_share_cols]
    #make a copy to avoid setting warnings
    copy_df = fcfe_per_share_df.copy(deep=True) 
    fcfe_per_share_df.loc[:,'fcfe_per_share_last_value'] = copy_df.apply(last_value, axis=1)
    fcfe_per_share_df.loc[:,'fcfe_per_share_last_year'] = copy_df.apply(lambda x: x.last_valid_index(), axis=1)
    fcfe_per_share_df.loc[:,'fcfe_per_share_first_value'] = copy_df.apply(first_value, axis=1)
    fcfe_per_share_df.loc[:,'fcfe_per_share_first_year'] = copy_df.apply(lambda x: x.first_valid_index(), axis=1)
    fcfe_per_share_df.loc[:,'fcfe_per_share_years_avail'] = copy_df.apply(lambda x: x.count(), axis =1)
    #massage year fields  
    fcfe_per_share_df[['fcfe_per_share_first_year', 'fcfe_per_share_last_year']] = fcfe_per_share_df[['fcfe_per_share_first_year', 'fcfe_per_share_last_year']].fillna(value=0)
    fcfe_per_share_df["fcfe_per_share_first_year"] = fcfe_per_share_df["fcfe_per_share_first_year"].str.replace("fcfe_per_share", "")
    fcfe_per_share_df["fcfe_per_share_first_year"].fillna(0, inplace = True)
    fcfe_per_share_df["fcfe_per_share_first_year"] = fcfe_per_share_df["fcfe_per_share_first_year"].astype(int, errors = "ignore")
    fcfe_per_share_df["fcfe_per_share_last_year"] = fcfe_per_share_df["fcfe_per_share_last_year"].str.replace("fcfe_per_share", "")
    fcfe_per_share_df["fcfe_per_share_last_year"].fillna(0, inplace = True)
    fcfe_per_share_df["fcfe_per_share_last_year"] = fcfe_per_share_df["fcfe_per_share_last_year"].astype(int, errors = "ignore")
   # print(fcfe_per_share_df.dtypes)
    #get years between first and lasts date 
    fcfe_per_share_df['fcfe_per_share_years_between'] = fcfe_per_share_df['fcfe_per_share_last_year'] - fcfe_per_share_df['fcfe_per_share_first_year']    
    #add ticker back
    fcfe_per_share_df.loc[:,'ticker'] = wide_df['ticker']   
    fcfe_per_share_df = fcfe_per_share_df[[
        'ticker',
        'fcfe_per_share_last_value',
        'fcfe_per_share_last_year',
        'fcfe_per_share_first_value',
        'fcfe_per_share_first_year',
        'fcfe_per_share_years_avail',
        'fcfe_per_share_years_between'
    ]]
    #print(fcfe_per_share_df)

    #join in stats 
    wide_df = wide_df.merge(revenue_df, on= ['ticker'], how = 'left')
    wide_df = wide_df.merge(shares_outstanding_df, on= ['ticker'], how = 'left')
    wide_df = wide_df.merge(fcfe_df, on= ['ticker'], how = 'left')
    wide_df = wide_df.merge(fcfe_per_share_df, on= ['ticker'], how = 'left')
    
    #print(wide_df)
    
    # Calculate FCFE Yields for each year ugh (since price comes in wide)
    ######################################################################################################
    calc_df = sqldf(
    """
    SELECT 
    *, 
    (IFNULL(fcfe_yield2013,0) +  IFNULL(fcfe_yield2014,0) +  IFNULL(fcfe_yield2015,0) +  
    IFNULL(fcfe_yield2016,0) +  IFNULL(fcfe_yield2017,0) +  IFNULL(fcfe_yield2018,0) +  
    IFNULL(fcfe_yield2019,0))/NULLIF(fcfe_per_share_years_avail,0) AS avg_fcfe_yield
    FROM 
    (
    SELECT 
    *, 
    fcfe_per_share2013/NULLIF(avg_price2013,0) AS fcfe_yield2013,
    fcfe_per_share2014/NULLIF(avg_price2014,0) AS fcfe_yield2014,
    fcfe_per_share2015/NULLIF(avg_price2015,0) AS fcfe_yield2015,
    fcfe_per_share2016/NULLIF(avg_price2016,0) AS fcfe_yield2016,
    fcfe_per_share2017/NULLIF(avg_price2017,0) AS fcfe_yield2017, 
    fcfe_per_share2018/NULLIF(avg_price2018,0) AS fcfe_yield2018,
    fcfe_per_share2019/NULLIF(avg_price2019,0) AS fcfe_yield2019 
    FROM 
    wide_df
    )
    """
    )
    
    #get growth measure 
    calc_df['revenue_CAGR']  = (calc_df['revenue_last_value']/calc_df['revenue_first_value']).fillna(0) ** (1/calc_df['revenue_years_between']).fillna(0) - 1
    calc_df['fcfe_per_share_CAGR']  = (calc_df['fcfe_per_share_last_value']/calc_df['fcfe_per_share_first_value']).fillna(0) ** (1/calc_df['fcfe_per_share_years_between']).fillna(0) - 1
    
    #set growth inhibitor factor 
    GIF = 0.75
    
    #get forecast components
    #what will market pay for future for fcfe
    calc_df['estimated_fcfe_multiplier'] = (1/calc_df['avg_fcfe_yield']).fillna(0) * GIF
    #how much will fcfe grow? 
    calc_df['estimated_fcfe_per_share_CAGR'] = calc_df['fcfe_per_share_CAGR'] * GIF
    
    
    #get estimated future fcfe 
    calc_df['estimated_fcfe_f1'] = calc_df['fcfe_per_share2019'] * (1+calc_df['estimated_fcfe_per_share_CAGR'])
    calc_df['estimated_fcfe_f2'] = calc_df['fcfe_per_share2019'] * (1+calc_df['estimated_fcfe_per_share_CAGR'])**2
    calc_df['estimated_fcfe_f3'] = calc_df['fcfe_per_share2019'] * (1+calc_df['estimated_fcfe_per_share_CAGR'])**3
    calc_df['estimated_fcfe_f4'] = calc_df['fcfe_per_share2019'] * (1+calc_df['estimated_fcfe_per_share_CAGR'])**4
    calc_df['estimated_fcfe_f5'] = calc_df['fcfe_per_share2019'] * (1+calc_df['estimated_fcfe_per_share_CAGR'])**5
    calc_df['estimated_fcfe_f6'] = calc_df['fcfe_per_share2019'] * (1+calc_df['estimated_fcfe_per_share_CAGR'])**6
    calc_df['estimated_fcfe_f7'] = calc_df['fcfe_per_share2019'] * (1+calc_df['estimated_fcfe_per_share_CAGR'])**7
    calc_df['estimated_fcfe_f8'] = calc_df['fcfe_per_share2019'] * (1+calc_df['estimated_fcfe_per_share_CAGR'])**8
    calc_df['estimated_fcfe_f9'] = calc_df['fcfe_per_share2019'] * (1+calc_df['estimated_fcfe_per_share_CAGR'])**9
    calc_df['estimated_fcfe_f10'] = calc_df['fcfe_per_share2019'] * (1+calc_df['estimated_fcfe_per_share_CAGR'])**10
    
    #estimated total cash in 
    calc_df['estimated_sale_price'] = calc_df['estimated_fcfe_f10'] * calc_df['estimated_fcfe_multiplier']
    calc_df['estimated_total_cfs'] = calc_df['estimated_sale_price'] - calc_df['current_price'] + calc_df['estimated_fcfe_f1'] + calc_df['estimated_fcfe_f2'] + calc_df['estimated_fcfe_f3'] + calc_df['estimated_fcfe_f4'] + calc_df['estimated_fcfe_f5'] + calc_df['estimated_fcfe_f6'] + calc_df['estimated_fcfe_f7'] + calc_df['estimated_fcfe_f8'] + calc_df['estimated_fcfe_f9'] + calc_df['estimated_fcfe_f10']
        
    calc_df['estimated_IRR'] = (calc_df['estimated_total_cfs']/calc_df['current_price']) ** (.1) -1     
    
    
    calc_df.set_index(keys= ['ticker'], inplace = True)
    ####################################################################################################### 
    #write to csv
    print("printing df to write to csv")
    print(calc_df)
    calc_df.to_csv("sp500_financials.csv")
    return(calc_df)
    
start = time.time()

print("starting to process data")
init_df = init_process_data()

end = time.time()
print("finished processing data")
print(end - start)

In [ ]:
for i in init_df.columns: print(i)


In [ ]:
good_guys = sqldf("""
SELECT 
DISTINCT 
ticker 
FROM 
init_df 
WHERE 
fcfe_per_share2019 IS NOT NULL 
AND fcfe_per_share2019 <> 0 
AND fcfe2019 <> 0
AND fcfe_per_share_years_avail > 2
AND fcfe_per_share_years_between > 1
      """)

for index, row in good_guys.iterrows():
    print(row['ticker'])

In [ ]:
# # section for manual data insertion
# df_edit = pd.read_csv("sp500_financials.csv")

# #index of dataframe should be ticker: ['ticker', 'colname'] = value

# #go thru sp500 alphabetically and set financials you want. 

# #ticker_mod = 'aapl'
# #df_edit.set_value(ticker_mod, 'revenue2013', 10)




# ########################################################################################################################
# #write to csv
# df_edit.to_csv("sp500_financials_editted.csv")

In [ ]:
#assess quality of data here 
pt_good = sqldf("""SELECT DISTINCT ticker FROM init_df 
                    ORDER BY cfo2019 DESC 
                """)
for index, row in pt_good.iterrows():
    print(row['ticker'])